In [1]:
import pandas as pd

In [2]:
# loading data master
from datetime import datetime

set_ = [1,2,3,4,5,6]
id_ = [1,2,3,4,5,6]

date = {
    1:[10,11],
    2:[12,13],
    3:[17,18],
    4:[19,20],
    5:[24,25],
    6:[26,27]
}

data_SkinT = pd.DataFrame(columns=['Time','Unit','1','2','3','4','Tmsk','experiment_set','experiment_id','subid','Thermal','Traffic noise levels','Speech'])
data_V4 = pd.read_excel("../0_dataset_master/Experiment_edited_V4.xlsx")
try:
    for set in set_ :
        for id in id_:
            data = pd.read_excel(f"../7_resampleIntoEachSet/SkinT_set{set}_adjustment.xlsx", sheet_name=f'idid{id}')
            data_fromV4 = data_V4[(data_V4['Experiment set']==f"set{set}") & (data_V4['Experiment id']==f"id{id}")]
            if data_fromV4.empty:
                continue
            list_of_date = []
            current_date = 0
            current_data = ""
            status = True
            counter = 0
            for ind, row in data_fromV4.iterrows():
                date = str(row["End Date"].date())
                start_clock, end_clock = row['Time_thermal'].split("-")
                start_time = date+" "+start_clock
                start_time = datetime.strptime(start_time, "%Y-%m-%d %H:%M")
                
                if status:
                    list_of_date.append(start_time)
                    status = False
                    current_date = datetime.strptime(date, "%Y-%m-%d").day

                if datetime.strptime(date, "%Y-%m-%d").day > current_date and not status:
                    list_of_date.append(current_data)
                    list_of_date.append(start_time)
                    current_date = datetime.strptime(date, "%Y-%m-%d").day

                end_time = date+" "+end_clock
                end_time = datetime.strptime(end_time, "%Y-%m-%d %H:%M")
                current_data = end_time

                if counter == len(list(data_fromV4.iterrows()))-1:
                    list_of_date.append(end_time)
                
                counter+=1
            data = data[((data['Time']>=list_of_date[0]) & (data['Time']<=list_of_date[1])) | ((data['Time']>=list_of_date[2]) & (data['Time']<=list_of_date[3]))]
            frames = [data_SkinT, data]
            data_SkinT = pd.concat(frames)
except ValueError as e:
    print(e)
    print(set, id)
except IndexError as e:
    print(e)
    print(set, id)
data_SkinT.dropna(subset=['Thermal', 'Traffic noise levels'], inplace=True)
data_SkinT = data_SkinT[data_SkinT['Speech']==0]

C:\Users\ZENI\AppData\Local\Temp\ipykernel_8216\3383596168.py:56: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  data_SkinT = pd.concat(frames)
C:\Users\ZENI\AppData\Local\Temp\ipykernel_8216\3383596168.py:56: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  data_SkinT = pd.concat(frames)


In [3]:
from datetime import datetime
result_tmsk_avg = []
try:
    for _, row in data_V4.iterrows():
        date = str(row["End Date"].date())
        start_clock, end_clock = row['Time_thermal'].split("-")
        start_time = date+" "+start_clock
        end_time = date+" "+end_clock

        end_time = datetime.strptime(end_time, "%Y-%m-%d %H:%M")
        start_time = datetime.strptime(start_time, "%Y-%m-%d %H:%M")

        dataFiltered = data_SkinT[(end_time > data_SkinT['Time']) & (start_time <= data_SkinT['Time']) & (data_SkinT['Experiment id']==row['Experiment id']) & (data_SkinT['Experiment set']==row['Experiment set'])]
        result_tmsk_avg.append(dataFiltered.Tmsk.mean(axis=0))
except ValueError:
    print(row) 

data_V4['Average_tmsk'] = result_tmsk_avg

In [4]:
from datetime import datetime
# loading data master
set_ = [1,2,3,4,5,6]
id_ = [1,2,3,4,5,6]

data_HR = pd.DataFrame(columns=['Time','HR','RR','MS','SKINCONTACT','experiment_set','experiment_id','subid','Thermal','Traffic noise levels','Speech'])
for set in set_ :
    for id in id_:
        data = pd.read_excel(f"../8_resampleHR/HR_set{set}.xlsx", sheet_name=f'id{id}')
        data_fromV4 = data_V4[(data_V4['Experiment set']==f"set{set}") & (data_V4['Experiment id']==f"id{id}")]
        if data_fromV4.empty:
                continue
        list_of_date = []
        current_date = 0
        current_data = ""
        status = True
        counter = 0
        for ind, row in data_fromV4.iterrows():
            date = str(row["End Date"].date())
            start_clock, end_clock = row['Time_thermal'].split("-")
            start_time = date+" "+start_clock
            start_time = datetime.strptime(start_time, "%Y-%m-%d %H:%M")
            
            if status:
                list_of_date.append(start_time)
                status = False
                current_date = datetime.strptime(date, "%Y-%m-%d").day

            if datetime.strptime(date, "%Y-%m-%d").day > current_date and not status:
                list_of_date.append(current_data)
                list_of_date.append(start_time)
                current_date = datetime.strptime(date, "%Y-%m-%d").day

            end_time = date+" "+end_clock
            end_time = datetime.strptime(end_time, "%Y-%m-%d %H:%M")
            current_data = end_time

            if counter == len(list(data_fromV4.iterrows()))-1:
                list_of_date.append(end_time)
            
            counter+=1
        data = data[((data['Time']>=list_of_date[0]) & (data['Time']<=list_of_date[1])) | ((data['Time']>=list_of_date[2]) & (data['Time']<=list_of_date[3]))]
        frames = [data_HR, data]
        data_HR = pd.concat(frames)
data_HR.dropna(subset=['Thermal', 'Traffic noise levels'], inplace=True)
data_HR = data_HR[data_HR['Speech']==0]
data_HR = data_HR[data_HR['HR']>0]
data_HR = data_HR[~((data_HR['Traffic noise levels']==45) & (data_HR['HR']>130))]

C:\Users\ZENI\AppData\Local\Temp\ipykernel_8216\3375172483.py:44: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  data_HR = pd.concat(frames)


In [24]:
from datetime import datetime
result_hr_avg = []
try:
    for _, row in data_V4.iterrows():
        date = str(row["End Date"].date())
        start_clock, end_clock = row['Time_thermal'].split("-")
        start_time = date+" "+start_clock
        end_time = date+" "+end_clock

        end_time = datetime.strptime(end_time, "%Y-%m-%d %H:%M")
        start_time = datetime.strptime(start_time, "%Y-%m-%d %H:%M")

        dataFiltered = data_HR[(end_time >= data_HR['Time']) & (start_time <= data_HR['Time']) & (data_HR['experiment_id']==int(row['Experiment id'][-1])) & (data_HR['experiment_set']==int(row['Experiment set'][-1]))]
        result_hr_avg.append(dataFiltered.HR.mean(axis=0))
except ValueError:
    print(row) 

data_V4['Average_HR'] = result_hr_avg

In [6]:
data_V4

,Unnamed: 0,No,Start Date,End Date,IP Address,Progress,Duration (in seconds),Finished,Recorded Date,Response ID,...,ThermalAcceptability,NoiseAnnoyance,AcousticSatisfaction,UnsatisfiedNoiseSource,UnsatisfiedNoiseSource - other,ThermalSatisfaction,TotalSatisfaction,SET,Average_tmsk,Average_HR
0,0,1,2023-07-10 10:18:12,2023-07-10 11:02:50,129.126.10.3,100,2677,True,2023-07-10 11:02:50,R_1QKIHfH2SzDYww5,...,3,2,4,NaN,NaN,6,4,26.5,33.569300,70.236576
1,1,2,2023-07-10 09:07:40,2023-07-10 11:03:41,129.126.10.6,100,6961,True,2023-07-10 11:03:42,R_3oZ1X5rJydNMWKi,...,4,1,6,NaN,NaN,6,6,26.5,33.726620,66.079414
2,2,3,2023-07-10 09:09:15,2023-07-10 11:03:52,129.126.10.8,100,6877,True,2023-07-10 11:03:53,R_3nvGWRYoFKSgygt,...,4,2,5,NaN,NaN,6,6,25.5,34.045650,70.002982
3,3,4,2023-07-10 10:18:07,2023-07-10 11:04:08,119.234.4.134,100,2761,True,2023-07-10 11:04:09,R_2U4WrS8qQBMNLbH,...,3,4,2,Other,Mouse clicking from the neighbouring table,5,3,25.8,33.803170,NaN
4,4,5,2023-07-10 10:45:42,2023-07-10 11:05:05,119.56.107.228,100,1163,True,2023-07-10 11:05:06,R_2EhPoNBA5xgmC4D,...,3,5,1,Equipment noise,NaN,5,1,26.1,32.733630,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301,301,477,2023-07-27 16:10:30,2023-07-27 16:27:35,119.56.99.96,100,1024,True,2023-07-27 16:27:35,R_2ttfFxeZl0jOvVq,...,3,3,3,Traffic noise,NaN,2,2,28.2,34.670600,78.538708
302,302,478,2023-07-27 15:47:28,2023-07-27 16:27:57,129.126.10.8,100,2428,True,2023-07-27 16:27:57,R_1dazsRpvRJKcZz8,...,2,2,5,NaN,NaN,2,3,28.8,34.686600,84.744517
303,303,479,2023-07-27 16:08:19,2023-07-27 16:28:20,129.126.10.14,100,1201,True,2023-07-27 16:28:20,R_2sXaFvpM2YYBSMn,...,3,2,3,Traffic noise,NaN,5,4,28.3,34.008805,81.522657
304,304,480,2023-07-27 15:38:12,2023-07-27 16:31:11,129.126.10.0,100,3178,True,2023-07-27 16:31:11,R_3Pd7YJJd4vSJ70N,...,2,2,3,Traffic noise,NaN,3,3,28.7,34.385950,74.242712


In [7]:
data_V4.to_excel("../0_dataset_master/Experiment_edited_V5.xlsx", index=False)